In [1]:
import io
import re
import time
import boto3

import requests
from bs4 import BeautifulSoup
import pandas as pd

from datetime import datetime, timedelta

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

### 주식 토론방 - 국내(from 네이버 금융)

In [2]:
korea = pd.read_csv('국내_산업분류.csv', encoding = 'cp949')
korea['종목코드'] = korea['종목코드'].str.replace("'", "")
korea

,소분류,기업,종목코드,시장
0,Specialty Chemicals,탑머티리얼,360070,KOSDAQ
1,Semiconductor Equipment & Materials,삼성전자,005930,KOSPI
2,Semiconductor Equipment & Materials,SK하이닉스,000660,KOSPI
3,Semiconductor Equipment & Materials,원익IPS,240810,KOSDAQ GLOBAL
4,Semiconductor Equipment & Materials,피에스케이홀딩스,031980,KOSDAQ
...,...,...,...,...
4280,Confectioners,빙그레,005180,KOSPI
4281,Confectioners,오뚜기,007310,KOSPI
4282,Confectioners,대한제분,001130,KOSPI
4283,Confectioners,대상,001680,KOSPI


In [3]:
# 날짜 기준
def naver_crawler(code):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}
    total_data = []
    
    # 날짜 설정
    end_date = datetime.today().date() - timedelta(days = 1)# 현재 날짜
    start_date = end_date - timedelta(days=7)  # 7일 전 날짜 계산

    page_num = 1
    while page_num <= 100:
        if page_num % 10 == 0:
            print(f'================== Page {page_num} is done ==================')
            
        url = f"https://finance.naver.com/item/board.nhn?code={str(code)}&page={str(page_num)}"
        result = requests.get(url, headers=headers)
        bs_obj = BeautifulSoup(result.content, "html.parser")
        table = bs_obj.find('table', {'class': 'type2'})
        if table :
            tt = table.select('tbody > tr')
        else :
            tt = []
            
        should_stop = False  # 데이터 수집 중단 여부

        for i in range(2, len(tt)):
            if len(tt[i].select('td > span')) > 0:
                date_str = tt[i].select('td > span')[0].text.split()[0]  # 시간 정보 제거
                date = datetime.strptime(date_str, "%Y.%m.%d").date()
                
                if date < start_date:
                    should_stop = True  # 데이터 수집 중단
                    break
                
                if date <= end_date:  # 7일 이내의 데이터만 수집
                    title = tt[i].select('td.title > a')[0]['title']
                    link = "https://finance.naver.com" + tt[i].select('td.title > a')[0]['href']
                    writer = tt[i].select('td.p11')[0].text.replace('\t', '').replace('\n', '')
                    views = tt[i].select('td > span')[1].text
                    pos = tt[i].select('td > strong')[0].text
                    neg = tt[i].select('td > strong')[1].text
                    commuData = {'날짜': date_str, '제목': title, '글쓴이': writer, '조회': views, '공감': pos, '비공감': neg, '내용_링크': link}
                    total_data.append(commuData)
                    
                    res = requests.get(link, headers=headers)
                    soup = BeautifulSoup(res.text, 'lxml')
                    commu_content = soup.select_one('#body').text.replace('\n', '').replace('\t', '').replace('\r', '')
                    commuData['내용'] = commu_content
                
        if should_stop:
            break
            
        page_num += 1

    total_df = pd.DataFrame(total_data)
    total_df['종목코드'] = code
    total_df['기업'] = korea.loc[korea['종목코드'] == code, '기업'].unique()[0]
    
    return total_df

In [4]:
def upload(data, comp, s3_client, bucket_name, endpoint_url, access_key, secret_key):
    # 데이터프레임을 파일로 저장
    xlsx_filename = f'{comp}_주식토론방.xlsx'
    xlsx_buffer = io.BytesIO()
    data.to_excel(xlsx_buffer, index = False)
    xlsx_buffer.seek(0)
    
    # xlsx 파일 클라우드 업로드
    object_name = f'kr_community_crawling/{xlsx_filename}'
    s3_client.upload_fileobj(xlsx_buffer, bucket_name, object_name)
    print(f"Uploaded {xlsx_filename} to S3")
    
def main():
    # AWS S3 설정
    service_name = 's3'
    endpoint_url = 'https://kr.object.ncloudstorage.com'
    access_key = 'cPKhn3d9z9ZWYMAPPwyq'
    secret_key = 'VFjEkpateO6VFGY0U9op073I4NXTlejMBMIq6FvA'
    bucket_name = 'bucket1chaegpt'
    
    # 국내 회사 리스트 생성
    kr_codes = korea['종목코드'].unique()
    
    # AWS S3 클라이언트 생성
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)
    # 클라우드 폴더 생성
    object_name = 'usa_community_crawling/'
    s3.put_object(Bucket=bucket_name, Key=object_name)
    
    for code in kr_codes:
        result = naver_crawler(code)
        comp = korea.loc[korea['종목코드'] == code, '기업'].unique()[0]
        
        upload(result, comp, s3, bucket_name, endpoint_url, access_key, secret_key)
        
if __name__ == '__main__':
    main()

================== Page 10 is done ==================
Uploaded 탑머티리얼_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 40 is done ==================
================== Page 50 is done ==================
================== Page 60 is done ==================
================== Page 70 is done ==================
================== Page 80 is done ==================
================== Page 90 is done ==================
================== Page 100 is done ==================
Uploaded 삼성전자_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 40 is done ==================
Uploaded SK하이닉스_주식토론방.xlsx to S3
Uploaded 원익IPS_주식토론방.xlsx to S3
Uploaded 피에스케이홀딩스_주식토론방.xlsx to S3
Uploaded 유진테크_주식토론방.xlsx t

C:\Users\pc\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://pubs.acs.org/doi/10.1021/acsenergylett.3c01651%20제안된%20실온%20초전도체%20LK-99의%20모화합물인%20납인회석회석은%20전하%20보상%20산화물%20이온을%20갖는%20%5bPb%2010%20(PO%204%20)%206%20%5d%20II%20지지체를%20특징으로%20합니다.%20이%20O%20&' with link or location/anchor > 2079 characters since it exceeds Excel's limit for URLS
  warn("Ignoring URL '%s' with link or location/anchor > %d "
C:\Users\pc\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://youtu.be/gHQ1LU6AJXc%20어매%20%20내%20%20타짜여.%20%20GAMBLER%20도박하는%20사람들이%20그냥%20잘%20%20찍음%20되지.무슨%20분석이여.운칠기삼~~다%20%20자기%208자대로.%20spac****%20성공시%20노벨상%20!!%20실패시%20즉시%20사형%20!!%20됐남.~~~~~%20%20jude****%20그냥%20강원랜드나%20%20가라.%20(홀%20%20&%20%20%20짝%20)둘%20중%20하나여.%20%20오늘도%20그랬듯이%20%20jyg2****댓글%20미국%20네이처지도%20안믿는%20나라는%20뭐다?신도들.~~오직%20%20교주님%20만%20%20믿습니다.%20hans****댓글%20한국%20초전도체%20연구%20역사는%2030년도%20안되었고%20미국,%20유럽,%20일본%20등%20기초과학%20선진국은%20100년

Uploaded LS전선아시아_주식토론방.xlsx to S3
Uploaded 아이엠_주식토론방.xlsx to S3
Uploaded 한빛소프트_주식토론방.xlsx to S3
Uploaded 에스코넥_주식토론방.xlsx to S3
Uploaded 이랜텍_주식토론방.xlsx to S3
Uploaded 에스피지_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 40 is done ==================
================== Page 50 is done ==================
================== Page 60 is done ==================
================== Page 70 is done ==================
================== Page 80 is done ==================
Uploaded 동운아나텍_주식토론방.xlsx to S3
Uploaded 와이제이엠게임즈_주식토론방.xlsx to S3
Uploaded 덱스터_주식토론방.xlsx to S3
Uploaded 나무가_주식토론방.xlsx to S3
================== Page 10 is done ==================


C:\Users\pc\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'http://m.g-enews.com/view.php?ud=20230827075152654e8b8a793f7_1%20%5b모닝픽%5d%20삼성과%20애플,%20반도체%20설계%20기업%20Arm의%20IPO에%20투자%20결정노정용%20기자%20◇삼성과%20애플,%20반도체%20설계%20기업%20Arm의%20IPO에%20투자%20결정%20애플과%20삼성은%20소프트뱅크의%20반도체%20설계%20기업인%20암(Arm)의%20기업공개(IPO)에%20투자하기로%20결정했다고%20더로지컬인디안이%2026일(현지%20시간)%20보도했다.%20이%20결정은%20단순히%20금전적%20이익을%20위한%20것이%20아니라,%20지속적으로%20변화하는%20기술%20시장에서%20경쟁%20우위를%20확보하기%20위한%20전략적인%20움직임으로%20평가된다.%20Arm의%20반도체%20설계는%20다양한%20디바이스에서%20중요한%20역할을%20하며,%20특히%20에너지%20효율적이고%20다양한%20기능을%20제공하는%20칩%20설계%20분야에서%20선두%20주자로%20자리매김하고%20있다.%205G%20네트워크,%20인공지능(AI),%20IoT%20디바이스의%20확산과%20같은%20현재의%20기술%20트렌드를%20감안할%20때%20Arm의%20칩%20설계%20기술은%20중요성을%20더욱%20높여간다.%20Apple과%20삼성의%20이런%20투자%20결정은%20두%20기업이%20제품%20혁신과%20시장%20리더십을%20유지하고자%20하는%20전략적%20노력의%20일환으로,%20Arm의%20기술과%20설계를%20통한%20산업%20간%20협업의%20잠재력을%20강조한다.%20기사입력%20:%202023-08-27%2007:56%20%20=================%20더로지컬인디안이%2026일(현지%20시간)%20

Uploaded 칩스앤미디어_주식토론방.xlsx to S3
Uploaded 아프리카TV_주식토론방.xlsx to S3
Uploaded 토탈소프트_주식토론방.xlsx to S3
================== Page 10 is done ==================
Uploaded 드래곤플라이_주식토론방.xlsx to S3
Uploaded 바른손이앤에이_주식토론방.xlsx to S3
Uploaded 세코닉스_주식토론방.xlsx to S3
Uploaded 지니뮤직_주식토론방.xlsx to S3
Uploaded 시공테크_주식토론방.xlsx to S3
Uploaded 미투온_주식토론방.xlsx to S3
Uploaded 다날_주식토론방.xlsx to S3
Uploaded 에이트원_주식토론방.xlsx to S3
Uploaded 위지윅스튜디오_주식토론방.xlsx to S3
Uploaded 제이씨현시스템_주식토론방.xlsx to S3
Uploaded 에스넷_주식토론방.xlsx to S3
Uploaded 큐에스아이_주식토론방.xlsx to S3
Uploaded 스코넥_주식토론방.xlsx to S3
Uploaded 비덴트_주식토론방.xlsx to S3
Uploaded 우리기술투자_주식토론방.xlsx to S3
Uploaded 매커스_주식토론방.xlsx to S3
Uploaded SGA_주식토론방.xlsx to S3
Uploaded KG모빌리언스_주식토론방.xlsx to S3
Uploaded 갤럭시아머니트리_주식토론방.xlsx to S3
Uploaded 티사이언티픽_주식토론방.xlsx to S3
Uploaded 위지트_주식토론방.xlsx to S3
Uploaded 에이티넘인베스트_주식토론방.xlsx to S3
Uploaded SBI인베스트먼트_주식토론방.xlsx to S3
Uploaded 한일진공_주식토론방.xlsx to S3
Uploaded 케이피엠테크_주식토론방.xlsx to S3
Uploaded 엠게임_주식토론방.xlsx to S3
Uploaded FSN_주식토론방

...

C:\Users\pc\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://m.stock.naver.com/fchart/foreign/stock/PFE%20고령자ㆍ임신부에%20사용%20허가...RSV%20하기도질환%20예방%20입증%5b의약뉴스%5d%20유럽%20집행위원회가%20미국%20식품의약국(FDA)에%20이어%20화이자의%20호흡기세포융합바이러스(RSV)%20백신%20아브리스보(Abrysvo)를%20고령자%20예방접종과%20영아%20보호를%20위한%20임신부%20예방접종%20백신으로%20허가했다.%20▲%20화이자의%20아브리스보는%20유럽에서%20고령자%20보호%20및%20임신부%20예방접종을%20통한%20영아%20보호를%20위해%20승인된%20최초이자%20유일한%20RSV%20백신이다.▲%20화이자의%20아브리스보는%20유럽에서%20고령자%20보호%20및%20임신부%20예방접종을%20통한%20영아%20보호를%20위해%20승인된%20최초이자%20유일한%20RSV%20백신이다.화이자는%20유럽%20집행위원회가%20고령자와%20함께%20임신부%20예방접종을%20통해%20영아를%20RSV%20질환으로부터%20보호할%20수%20있는%202가%20RSV%20융합%20전%20F(RSVpreF)%20백신인%20아브리스보를%20판매%20허가했다고%2024일(현지시각)%20발표했다.%20유럽에서%20아브리스보는%20△%20임신%20중%20모성%20예방접종%20후%20출생부터%20생후%206개월까지%20영아에서%20RSV로%20인한%20하기도질환(LRTD)에%20대한%20수동%20방어%20△%2060세%20이상에서%20RSV로%20인한%20하기도질환%20예방을%20위한%20능동면역%20용도로%20승인됐다.%20이%20판매%20허가는%20앞서%20지난달에%20유럽의약품청(EMA)의%20약물사용자문위원회(CHMP)가%20제시한%20긍정적인%20의견에%20따른%2

Uploaded 바이오니아_주식토론방.xlsx to S3
Uploaded 진매트릭스_주식토론방.xlsx to S3
Uploaded 일성신약_주식토론방.xlsx to S3
Uploaded 보령_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
Uploaded 씨젠_주식토론방.xlsx to S3
================== Page 10 is done ==================
Uploaded 진원생명과학_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
Uploaded 씨티씨바이오_주식토론방.xlsx to S3
Uploaded 고려제약_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 40 is done ==================
================== Page 50 is done ==================
================== Page 60 is done ==================
================== Page 70 is done ==================
================== Page 80 is done ==================


C:\Users\pc\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://www.nbcnews.com/news/us-news/kentucky-school-district-cancels-classes-covid-flu-strep-rcna101511%20켄터키%20교육구는%20코비드,%20독감,%20연쇄상구균%20발병으로%20인해%20일년에%202주도%20채%20안되어%20수업을%20취소합니다%202023년%208월%2024일%20오전%208시(KST)마이아%20판데이(%20Maia%20Pandey)학년도가%20시작된%20지%202주도%20채%20안%20되어%20켄터키주의%20한%20교육구는%20학생의%20거의%205분의%201이%20코비드,%20패혈성%20인두염,%20독감%20및%20기타%20질병에%20걸린%20후%20나머지%20주%20동안%20대면%20수업을%20취소했습니다.%20학생%20수가%20900명%20미만인%20Lee%20카운티%20교육구는%208월%209일에%20수업을%20시작했지만%20금요일에는%20출석률이%20약%2082%25로%20감소했다고%20Earl%20Ray%20Schuler%20교육감이%20말했습니다.%20%20월요일까지%20그%20비율은%2081%25로%20떨어졌고%2014명의%20직원이%20병가를%20냈다고%20Schuler는%20말했습니다.%20교육구는%20화요일과%20수요일에%20수업을%20취소%20하고%20목요일과%20금요일에%20원격%20학습으로%20전환합니다.%20Schuler는%20학교를%20대대적으로%20청소하기%20위해%20스포츠%20연습과%20게임을%20포함한%20과외%20활동을%20이번%20주%20내내%20취소했다고%20말했습니다.%20“우리는%20버스와%20건물을%20소독하고%20직원과%20학생들에게%20치료할%20시간을%20제공하고%20있습니다.”라고%20그는%20말했습니다.%20켄터키주%20비티빌에

Uploaded 신풍제약_주식토론방.xlsx to S3
Uploaded 대성미생물_주식토론방.xlsx to S3
Uploaded 코미팜_주식토론방.xlsx to S3
Uploaded 엑서지21_주식토론방.xlsx to S3
Uploaded 제일바이오_주식토론방.xlsx to S3
Uploaded 체시스_주식토론방.xlsx to S3
================== Page 10 is done ==================
Uploaded 엑세스바이오_주식토론방.xlsx to S3
Uploaded 아이진_주식토론방.xlsx to S3
Uploaded 비엘_주식토론방.xlsx to S3
Uploaded 진바이오텍_주식토론방.xlsx to S3
Uploaded 피씨엘_주식토론방.xlsx to S3
Uploaded SK케미칼_주식토론방.xlsx to S3
================== Page 10 is done ==================
Uploaded 옵티팜_주식토론방.xlsx to S3
Uploaded 광동제약_주식토론방.xlsx to S3
Uploaded 수젠텍_주식토론방.xlsx to S3
Uploaded 바디텍메드_주식토론방.xlsx to S3
================== Page 10 is done ==================
Uploaded 미코바이오메드_주식토론방.xlsx to S3
Uploaded 경남제약_주식토론방.xlsx to S3
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
Uploaded SK바이오사이언스_주식토론방.xlsx to S3
Uploaded 차백신연구소_주식토론방.xlsx to S3
Uploaded 롯데쇼핑_주식토론방.xlsx to S3
Uploaded 대구백화점_

KeyboardInterrupt: 